In [27]:
import gamspy as gp
import gamspy.math as gpm
from gamspy import Sum, Card

import sys
import numpy as np
import pandas as pd

In [29]:
drone = pd.read_csv("drone.csv")
loc = pd.read_csv("location.csv")

drone_df = pd.DataFrame(drone)
loc_df = pd.DataFrame(loc)

FileNotFoundError: [Errno 2] No such file or directory: 'drone.csv'

In [ ]:
locations = list(loc_df["Location"])
models = list(drone_df["Drone Models"])
loc_df.index = locations
drone_df.index = models
loc_df = loc_df.drop(columns = "Location")
drone_df = drone_df.drop(columns = "Drone Models")
display(loc_df)
display(drone_df)

,Latitude,Longitude,Payload
Camp Randall,43.070227,-89.412687,10.0000
Sam's Apartment,43.075462,-89.393407,2.6250
3006 Sunrise Ct,43.108656,-89.473928,5.0000
Kohl Center,43.069709,-89.396907,4.6875
Computer Sciences Building,43.071286,-89.406561,6.1700
Noland Hall,43.071678,-89.404489,1.5000
Lake Mendota,43.107811,-89.419515,4.0000
The Kollege Klub,43.075951,-89.397279,11.2500
Tenney Park,43.092186,-89.367120,2.2000
The Bakke,43.076852,-89.420159,3.4500


,Carrying Capacity,Fly in Rain,Speed,Radius,Charging Current
MK30,5,1,80.47,32.1,840
MK27,5,0,80.47,16.1,560
